In [ ]:
import numpy as np
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

# Algoritmo Genético Binário em Passos:

1. **Representação binária**: Cada indivíduo da população é representado como um vetor binário.

2. **Função objetivo**: Avaliar cada indivíduo de acordo com a função de aptidão (fitness).

3. **Seleção**: Selecionar indivíduos com maior aptidão para reprodução.

4. **Cruzamento (crossover)**: Combinar partes dos bits de dois indivíduos (pais) para gerar novos indivíduos (filhos).

5. **Mutação**: Alterar aleatoriamente alguns bits dos indivíduos para manter diversidade.

6. **Convergência**: Repetir até atingir um critério de parada (número de gerações ou convergência).

## Gerar a População Inicial

In [ ]:
def generate_pop(nvar, intervals, popsize):
  pop = []
  if (nvar != len(intervals)):
    raise ValueError("Número de variáveis não corresponde ao número de intervalos")

  for i in range(nvar):
    pop.append(np.random.uniform(low=intervals[i][0], high=intervals[i][1], size=popsize))

  return np.array(pop).T

## Representação Binária

In [ ]:
def normalize_pop(pop, intervals):
  for i in range(pop.shape[1]):
    pop[:, i] = (pop[:, i] - intervals[i][0]) / (intervals[i][1] - intervals[i][0])
  return pop

In [ ]:
def unnormalize_pop(pop, intervals):
  for i in range(pop.shape[1]):
    pop[:, i] = pop[:, i] * (intervals[i][1] - intervals[i][0]) + intervals[i][0]
  return pop

In [ ]:
def ga_encode(par, intervals, nbitg):
    popsize, nvar = par.shape  # Tamanho da população e número de variáveis

    # Normaliza os valores contínuos para [0, 1]
    norm_par = normalize_pop(par, intervals)

    # Multiplica pelos limites do número de bits para representar em binário
    scaled_par = (norm_par * (2**nbitg - 1)).astype(int)

    # Codifica cada valor contínuo em binário, para cada variável
    bin_rep = np.array([[list(np.binary_repr(scaled_par[i, j], nbitg))
                         for j in range(nvar)] for i in range(popsize)]).astype(int)

    # Concatena os bits de todas as variáveis
    population = bin_rep.reshape(popsize, nvar * nbitg)

    return population

In [ ]:
def ga_decode(population, intervals, nbitg):
    popsize, nbitc = population.shape  # Tamanho da população e número de bits por cromossomo
    nvar = nbitc // nbitg  # Número de variáveis por indivíduo

    # Calcula o peso binário para conversão
    quant = 0.5 ** np.arange(1, nbitg + 1)
    quant /= sum(quant)  # Normaliza

    # Decodifica a população binária em valores contínuos
    par = np.dot(population.reshape(popsize, nvar, nbitg), quant)  # Combina bits

    # Escala para os limites a e b
    par = unnormalize_pop(par, intervals)

    return par

## Função Objetivo

In [ ]:
def fitness_function(x):
    y = (x[0]**2 + x[1] - 11)**2 + (x[0] + (x[1]**2) - 7)**2
    return y

## Seleção

In [ ]:
def select_parents(pop_bin, intervals, nbitg, mode, percent_kept=0.5):
  if (mode == 'roulette'):
    pop = ga_decode(pop_bin.copy(), intervals, nbitg)

    fitness = np.apply_along_axis(fitness_function, 1, pop)

    constante_aditiva = abs(np.min(fitness)) + 1
    fitness_ajustados = fitness + constante_aditiva

    total_fitness = np.sum(fitness_ajustados)
    probabilities = fitness_ajustados / total_fitness

    size = int(pop.shape[0] * percent_kept)
    parents = np.random.choice(pop_bin.shape[0], size=size, p=probabilities, replace=False)

    new_pop = np.zeros(shape=pop_bin.shape)
    new_pop[0: size] = pop_bin[parents]

    return new_pop, size

## Cruzamento

In [ ]:
def crossover(pop_bin, index):
  mew_pop = pop_bin.copy()

  for i in range(index, pop_bin.shape[0]):
    parents = np.random.randint(0, index, size=2)
    cross_point = np.random.randint(0, pop_bin.shape[1])
    mew_pop[i, 0:cross_point] = pop_bin[parents[0], 0:cross_point]
    mew_pop[i, cross_point:] = pop_bin[parents[1], cross_point:]

  return mew_pop

## Mutação

In [ ]:
def mutacao_vetorizada(pop, pmut):
    # Cria uma máscara booleana com a mesma forma da população
    mascara_mutacao = np.random.rand(*pop.shape) < pmut

    # Inverte os bits onde a máscara é True
    populacao_mutada = pop.copy()
    populacao_mutada[mascara_mutacao] = 1 - populacao_mutada[mascara_mutacao]

    return populacao_mutada

## Substituição

In [ ]:
def substitute(pop_ant, pop_prox, intervals, nbitg, prob):
  pop_ant_fitness = np.apply_along_axis(fitness_function, 1, ga_decode(pop_ant.copy(), intervals, nbitg))
  pop_prox_fitness = np.apply_along_axis(fitness_function, 1, ga_decode(pop_prox.copy(), intervals, nbitg))

  pop_ant_fit = np.column_stack((pop_ant, pop_ant_fitness))
  pop_prox_fit = np.column_stack((pop_prox, pop_prox_fitness))

  if prob == 'max':
    pop_ant_sort = pop_ant_fit[pop_ant_fit[:, -1].argsort()[::-1]]
    pop_prox_sort = pop_prox_fit[pop_prox_fit[:, -1].argsort()[::-1]]
  else:
    pop_ant_sort = pop_ant_fit[pop_ant_fit[:, -1].argsort()]
    pop_prox_sort = pop_prox_fit[pop_prox_fit[:, -1].argsort()]

  pop_ant_sort = pop_ant_sort[:, :-1]
  pop_prox_sort = pop_prox_sort[:, :-1]

  new_pop = np.zeros(shape=pop_ant.shape)
  new_pop[0] = pop_ant_sort[0]
  new_pop[1:] = pop_prox_sort[:-1]

  return new_pop

## Solução

In [ ]:
def best_individual(pop, params):
  pop_decode = ga_decode(pop.copy(), params['intervals'], params['nbitg'])
  pop_fitness = np.apply_along_axis(fitness_function, 1, pop_decode)
  if params['prob'] == 'max':
    best_index = np.argmax(pop_fitness)
  else:
    best_index = np.argmin(pop_fitness)

  best_individual = pop_decode[best_index]
  best_fitness = pop_fitness[best_index]

  # print(f"Melhor indivíduo: {best_individual}")
  # print(f"Fitness do melhor indivíduo: {best_fitness}")
  return best_individual

## Loop

In [ ]:
def ga_bin(params):
  initial_pop = generate_pop(params['nvar'], params['intervals'], params['popsize'])
  pop_bin = ga_encode(initial_pop, params['intervals'], params['nbitg'])
  bests = [best_individual(pop_bin, params)]

  for iter in range(params['max_iter']):
    parents, index = select_parents(pop_bin.copy(), params['intervals'], params['nbitg'], 'roulette', params['percent_kept'])
    mating_pool = crossover(parents, index)
    mating_pool_mut = mutacao_vetorizada(mating_pool, params['pmut'])

    new_pop = substitute(pop_bin, mating_pool_mut, params['intervals'], params['nbitg'], params['prob'])
    pop_bin = new_pop
    bests.append(best_individual(pop_bin, params))

  return np.array(bests)

## Execução

In [ ]:
params = {
    'nvar': 2,
    'popsize': 100,
    'intervals': [[-10, 10], [-10, 10]],
    'nbitg': 16,
    'percent_kept': 0.5,
    'pmut': 0.05,
    'max_iter': 100,
    'prob': 'min'
}

In [ ]:
bests = ga_bin(params)

## Plots

In [ ]:
# Gerar dados para o gráfico
x_range = np.linspace(*params['intervals'][0], 100)
y_range = np.linspace(*params['intervals'][1], 100)
x_mesh, y_mesh = np.meshgrid(x_range, y_range)

z_mesh = fitness_function([x_mesh, y_mesh])

points = np.column_stack((bests, np.apply_along_axis(fitness_function, 1, bests)))  # Exemplo de dados aleatórios para ilustrar

# Separar os valores de X, Y, Z
x_points = points[:, 0]
y_points = points[:, 1]
z_points = points[:, 2]

# Criar o gráfico de superfície 3D
surface = go.Surface(x=x_mesh, y=y_mesh, z=z_mesh, opacity=0.7)

# Adicionar os pontos à superfície usando Scatter3d
scatter_points = go.Scatter3d(
    x=x_points,
    y=y_points,
    z=z_points,
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Points'
)

# Layout do gráfico
layout = go.Layout(title="Fitness Function 3D Plot with Points",
                   scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Fitness Value'))

# Combinar o gráfico de superfície com os pontos
fig = go.Figure(data=[surface, scatter_points], layout=layout)
fig.show()